"""This is an example of using Hierarchical RNN (HRNN) to classify MNIST digits.
HRNNs can learn across multiple levels of temporal hierarchy over a complex sequence.
Usually, the first recurrent layer of an HRNN encodes a sentence (e.g. of word vectors)
into a  sentence vector. The second recurrent layer then encodes a sequence of
such vectors (encoded by the first layer) into a document vector. This
document vector is considered to preserve both the word-level and
sentence-level structure of the context.
#References
    - [A Hierarchical Neural Autoencoder for Paragraphs and Documents](https://arxiv.org/abs/1506.01057)
        Encodes paragraphs and documents with HRNN.
        Results have shown that HRNN outperforms standard
        RNNs and may play some role in more sophisticated generation tasks like
        summarization or question answering.
    - [Hierarchical recurrent neural network for skeleton based action recognition](http://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=7298714)
        Achieved state-of-the-art results on skeleton based action recognition with 3 levels
        of bidirectional HRNN combined with fully connected layers.
In the below MNIST example the first LSTM layer first encodes every
column of pixels of shape (28, 1) to a column vector of shape (128,). The second LSTM
layer encodes then these 28 column vectors of shape (28, 128) to a image vector
representing the whole image. A final Dense layer is added for prediction.
After 5 epochs: train acc: 0.9858, val acc: 0.9864
"""

In [1]:

from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, Dense, TimeDistributed
from keras.layers import LSTM

from keras_tqdm import TQDMNotebookCallback

import pickle

Using MXNet backend.
/anaconda/envs/py35/lib/python3.5/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL


In [2]:


# Training parameters.
batch_size = 32
num_classes = 10
epochs = 5

# Embedding dimensions.
row_hidden = 128
col_hidden = 128

# The data, shuffled and split between train and test sets.
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshapes data to 4D for Hierarchical RNN.
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')









x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [3]:
# Converts class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

AttributeError: module 'keras.utils' has no attribute 'to_categorical'

In [7]:
output_file='./pckl/mnist_hier_rnn_y_train.pckl'
f = open(output_file, 'wb')
pickle.dump(y_train, f)
f.close()

output_file='./pckl/mnist_hier_rnn_y_test.pckl'
f = open(output_file, 'wb')
pickle.dump(y_test, f)
f.close()


In [4]:
output_file='./pckl/mnist_hier_rnn_y_train.pckl'
f = open(output_file, 'rb')
y_train = pickle.load(f)
f.close()

output_file='./pckl/mnist_hier_rnn_y_test.pckl'
f = open(output_file, 'rb')
y_test = pickle.load(f)
f.close()

In [5]:
row, col, pixel = x_train.shape[1:]

# 4D input.
x = Input(shape=(row, col, pixel))

# Encodes a row of pixels using TimeDistributed Wrapper.
encoded_rows = TimeDistributed(LSTM(row_hidden))(x)

# Encodes columns of encoded rows.
encoded_columns = LSTM(col_hidden)(encoded_rows)

# Final predictions and model.
prediction = Dense(num_classes, activation='softmax')(encoded_columns)
model = Model(x, prediction)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'],context=['gpu(0)'])

In [3]:
#TensorFlow
print("Keras Backend: ", keras.backend.backend())
print("Epochs: ", epochs)
# Training.
%timeit -r 2 -n 1 model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=0,validation_data=(x_test, y_test),callbacks=[TQDMNotebookCallback(leave_inner=True, leave_outer=True)])

# Evaluation.
scores = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Keras Backend:  tensorflow
Epochs:  5



14min 55s ± 803 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
Test loss: 0.0424380446022
Test accuracy: 0.9878


In [3]:
#CNTK
print("Keras Backend: ", keras.backend.backend())
print("Epochs: ", epochs)
# Training.
%timeit  -r 2 -n 1 model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=0,validation_data=(x_test, y_test),callbacks=[TQDMNotebookCallback(leave_inner=True, leave_outer=True)])

# Evaluation.
scores = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Keras Backend:  cntk
Epochs:  5


/anaconda/envs/py35/lib/python3.5/site-packages/cntk/core.py:361: UserWarning: your data is of type "float64", but your input variable (uid "Input1272") expects "<class 'numpy.float32'>". Please convert your data beforehand to speed up training.
  (sample.dtype, var.uid, str(var.dtype)))


59968/|/[loss: 0.029, acc: 0.992] 100%|| 59968/60000 [02:44<00:00, 390.51it/s]
13min 41s ± 67.8 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
Test loss: 0.0525621209706
Test accuracy: 0.9859


In [ ]:
#Theano
print("Keras Backend: ", keras.backend.backend())
print("Epochs: ", epochs)
# Training.
%timeit  -r 2 -n 1 model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=0,validation_data=(x_test, y_test),callbacks=[TQDMNotebookCallback(leave_inner=True, leave_outer=True)])

# Evaluation.
scores = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [ ]:
#MXNet
print("Keras Backend: ", keras.backend.backend())
print("Epochs: ", epochs)
# Training.
%timeit  -r 2 -n 1 model.fit(x_train, y_train,batch_size=batch_size,nb_epoch=epochs,verbose=0,validation_data=(x_test, y_test),callbacks=[TQDMNotebookCallback(leave_inner=True, leave_outer=True)])

# Evaluation.
scores = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Keras Backend:  mxnet
Epochs:  5


/anaconda/envs/py35/lib/python3.5/site-packages/mxnet/module/bucketing_module.py:385: UserWarning: Optimizer created manually outside Module but rescale_grad is not normalized to 1.0/batch_size/num_workers (1.0 vs. 0.03125). Is this intended?
  force_init=force_init)


59968/|/[loss: 0.314, acc: 0.899] 100%|| 59968/60000 [02:40<00:00, 416.02it/s]

59968/|/[loss: 0.098, acc: 0.971] 100%|| 59968/60000 [02:38<00:00, 418.81it/s]

59968/|/[loss: 0.072, acc: 0.979] 100%|| 59968/60000 [02:38<00:00, 416.73it/s]

59968/|/[loss: 0.056, acc: 0.984] 100%|| 59968/60000 [02:38<00:00, 414.51it/s]

59968/|/[loss: 0.049, acc: 0.985] 100%|| 59968/60000 [02:38<00:00, 416.59it/s]


59968/|/[loss: 0.042, acc: 0.987] 100%|| 59968/60000 [02:40<00:00, 414.83it/s]

In [ ]:
# Evaluation.
scores = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])